# Radial Velocity Tutorial for MCMC

By Roberto Tejada (2019)

This tutorial will assume the user is familiar with the `Driver` class and is acquainted with MCMC terminology. For more information about MCMC, see the [MCMC Introduction Tutorial](https://orbitize.readthedocs.io/en/latest/tutorials/MCMC_tutorial.html). 

Here we explain how to fit radial velocity data using the MCMC technique. First we need a set of data containing radial velocity. We check the  data using `read_input` and observe the `quant_type` column for rv data. For more information on `orbitize.read_input.read_file()`, see the [Formatting Input Tutorial](https://orbitize.readthedocs.io/en/latest/formatting_inputs.html). 



## Import Modules and see rv data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import orbitize
from orbitize import read_input, system, priors, sampler, driver
import multiprocessing as mp
import pprint

testdata_path = '/home/sda/rtejada/orbitize/tests/' #your test data path here (change later to format)
data_table = read_input.read_file("{}/rv_testdata.csv".format(testdata_path)) #print all columns
data_table.pprint_all()

Exception: Unable to read file: /home/sda/rtejada/orbitize/tests//rv_testdata.csv. 
 Please check file path and format.

The `quant_type` column displays the type of data each row contains: astrometry (radec) or radial velocity (rv). For astrometry, `quant1` column contains right ascension and the `quant2` column contains declination. For rv data, `quant1` contains radial velocity data, while `quant2` is filled with `NaN`. The table contains each respective error column. 

We can now initialize the `Driver` class. MCMC samplers take time to converge to absolute maxima in parameter space, and the more parameters we introduce, the longer we expect it to take.

## Create Driver Object

For orbit fits using RV data, it is always necessary to fit the primary and secondary masses (`m0` and `m1`respectively as separate free parameters. This differs from the astrometry-only case where fitting the total mass `mtot` suffices. We must therefore set the system keyword `fit_secondary_mass` to `True` when initializing the Driver object.

In [ ]:
filename = "{}/rv_testdata.csv".format(testdata_path)

# system parameters
num_secondary_bodies = 1
system_mass = 1.97 # [Msol]
plx = 57.78 # [mas]
mass_err = 0.05 # [Msol]
plx_err = 0.05 # [mas]

# MCMC parameters
num_temps = 5
num_walkers = 30
num_threads = mp.cpu_count() # or a different number if you prefer


my_driver = driver.Driver(
    filename, 'MCMC', num_secondary_bodies, system_mass, plx, mass_err=mass_err, plx_err=plx_err,
    system_kwargs = {'fit_secondary_mass':True,
                                  'tau_ref_epoch':0},
    mcmc_kwargs={'num_temps': num_temps, 'num_walkers': num_walkers, 'num_threads': num_threads}
)

Since MCMC is class object in orbitize, we can assign a variable to the sampler and work with this variable like so:

In [ ]:
m = my_driver.sampler

# RV Priors

The priors for the two radial velocity parameters, the radial velocity offset (gamma), and jitter (sigma), have default uniform prior and log uniform prior respectively. The gamma uniform prior is set between $(-5,5)$ km/s, and the jitter log uniform prior is set for ($10^{-4},0.05$) km/s. In `system.System`, we define the parameter `m1` as the companion mass and `m0` as the stellar mass. The prior for `m1` is a log uniform prior and is set for ($10^{-3},2.0$) The current version of orbitize addressed in this tutorial returns the stellar radial velocity only. 

We can change the priors as instructed in the [Modifying Priors](https://orbitize.readthedocs.io/en/latest/tutorials/Modifying_Priors.html) tutorial:

In [ ]:
# getting the system object:

sys = my_driver.system

lab = sys.param_idx

print(sys.labels)
print(sys.sys_priors)

print(vars(sys.sys_priors[lab['m1']]))

In [ ]:
# change the m1 prior:

sys.sys_priors[lab['m1']] = priors.LogUniformPrior(1e-4, 0.5)

print(sys.labels)
print(sys.sys_priors)
print(vars(sys.sys_priors[lab['m1']]))

## Running the MCMC Sampler

As noted in the [MCMC Introduction Tutorial](https://orbitize.readthedocs.io/en/latest/tutorials/MCMC_tutorial.html), we must choose the sampler step for MCMC and save every nth sample to avoid using too much disk space.

In [ ]:
total_orbits = 10000 # number of steps x number of walkers (at lowest temperature)
burn_steps = 10 # steps to burn in per walker
thin = 10 # only save every 10th step

In [ ]:
m.run_sampler(total_orbits, burn_steps=burn_steps, thin=thin)

Now we can plot the distribution of MCMC parameter of interest:

In [ ]:
accepted_m1 = m.results.post[:, lab['m1']]
plt.hist(accepted_m1)
plt.xlabel('m1'); plt.ylabel('number of orbits')
plt.show()

# Saving Results over Extended MCMC Run

Sometimes our MCMC run will need to run for an extended period of time to let the walkers converge. To observe the convergence, we often need to see the walkers' progress along parameter space. We can save the sampler results periodically and keep running the sampler until convergence. Therefore, if we want to run for a greater number of steps and periodically save the results, we can create a for-loop and run for as many iterations as we'd like.  

We define a new driver object for this loop as follows:

In [ ]:
# MCMC parameters
num_temps = 5
num_walkers = 30
num_threads = mp.cpu_count() # or a different number if you prefer

n_iter = 2 # number of iterations

my_driver = driver.Driver(
    filename, 'MCMC', num_secondary_bodies, system_mass, plx, mass_err=mass_err, plx_err=plx_err,
    system_kwargs = {'fit_secondary_mass':True,
                                  'tau_ref_epoch':0},
    mcmc_kwargs={'num_temps': num_temps, 'num_walkers': num_walkers, 'num_threads': num_threads}
)

m = my_driver.sampler

We defined the number iterations (`n_iter`), or the number of iterations the loop will run, and we defined `orbits_per_iter`, which defines the number of steps per iteration. We also want a place to save our results per each iteration, so we create a directory called `orbitize_results/mcmcrun`. The subfulder is a timestamp of the time we run our sampler formated as (year-month-day-hour-minute-second). Keeping track of the hours and seconds makes sure we do not create directories that already exist if we run the cell below more than once:

In [ ]:
import os
from datetime import datetime

today = datetime.now()
path = '/home/sda/rtejada/' #update this path later 

directory = path + 'orbitize_results/mcmcrun/' + today.strftime('%Y-%m-%d-%H-%M-%S') + '/'
os.makedirs(directory)

We're now ready for the loop! The `results` object contains a `save_results` function which lets us save the results for our directory:

In [ ]:
for i in range(n_iter):
    #running the sampler:
    orbits = m.run_sampler(total_orbits, burn_steps=burn_steps, thin=thin)
    results = m.results
    results.save_results(directory + 'orbits_rv_%1d' % i) #saves results object as an hdf5 file.

# Plotting and Accesing Saved Results

We can plot the corner plot saved in the results object by following the steps in the [Advanced Plotting Tutorial](https://orbitize.readthedocs.io/en/latest/tutorials/Plotting_tutorial.html):

In [ ]:
corner_figure = results.plot_corner(quantiles=[0.16,0.5,0.84],show_titles=True,title_kwargs={"fontsize": 17})

As illustrated in the plot above, MCMC needs more time to run. We only performed two iterations in the loop to demonstrate its useage, but with increased `n_iter`, the trendplots saved in the loop and the corner plot will show how the walkers converge to absolute extrema in parameter space.

To access the saved data, we can read it into a results object as shown in the [MCMC Introduction Tutorial](https://orbitize.readthedocs.io/en/latest/tutorials/MCMC_tutorial.html):

In [ ]:
from orbitize import results
hdf5_filename = directory + 'orbits_rv_1'
loaded_results = results.Results() # Create blank results object for loading
loaded_results.load_results(hdf5_filename)

To demonstrate use of the loaded results file above, we can use the saved results to plot our orbital plots:

In [ ]:
epochs = my_driver.system.data_table['epoch']
orbit_plot_fig = loaded_results.plot_orbits(
    object_to_plot = 1, # Plot orbits for the first (and only, in this case) companion
    num_orbits_to_plot= 50, # Will plot 50 randomly selected orbits of this companion
    start_mjd=epochs[0] # Minimum MJD for colorbar (here we choose first data epoch)
)